In [33]:
import os
import tensorflow_hub as hub
import cv2
import numpy
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib

In [22]:
# Loading model directly from TensorFlow Hub
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite4/detection/2")

# Loading csv with labels of classes
labels = pd.read_csv('labels.csv', sep=';', index_col='ID')
labels = labels['OBJECT (2017 REL.)']

In [123]:
# Creating prediction
def pred(img_name, width = 1024, height = 1024):
    #Load image by Opencv2
    img = cv2.imread('rolex/' +  img_name)
    if img.size > 0:
        inp = cv2.resize(img, (width , height) )
        #Convert img to RGB
        rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)
        # COnverting to uint8
        rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)
        #Add dims to rgb_tensor
        rgb_tensor = tf.expand_dims(rgb_tensor , 0)
    
    
        boxes, scores, classes, num_detections = detector(rgb_tensor)

        # Processing outputs
        pred_labels = classes.numpy().astype('int')[0]
        pred_labels = [labels[i] for i in pred_labels]
        pred_boxes = boxes.numpy()[0].astype('int')
        pred_scores = scores.numpy()[0]

        # Putting the boxes and labels on the image
        for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
            if score > 0.5 and label == 'clock':
                score_txt = f'{100 * round(score)}%'
                #img_boxes = cv2.rectangle(rgb,(xmin, ymax),(xmax, ymin),(0,255,0),2)
                #font = cv2.FONT_HERSHEY_SIMPLEX
                #cv2.putText(img_boxes, label,(xmin, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
                #cv2.putText(img_boxes,score_txt,(xmax, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
                try:
                    cv2.imwrite('rolex_out/' + img_name, inp[int(ymin*0.9):int(ymax*1.1), int(xmin*0.9):int(xmax*1.1)])
                except:
                    cv2.imwrite('rolex_out/' + img_name, inp)



In [124]:
for img in os.listdir('rolex'):
    pred(img) 

AttributeError: 'NoneType' object has no attribute 'size'